In [2]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
device = torch.device("cuda")
    
    
batch_size = 128
model_name = "Resnet_20_quant_project"
model = resnet20_quant_project()


normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
print_freq = 100
def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch,adjust_list):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1

In [5]:
lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,weight_decay=weight_decay)

if not os.path.exists('result'): 
    os.makedirs('result')

fdir = 'result/'+str(model_name) 
if not os.path.exists(fdir):
    os.makedirs(fdir)

adjust_list = [80,90]

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch,adjust_list)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.723 (0.723)	Data 0.588 (0.588)	Loss 2.4852 (2.4852)	Prec 7.031% (7.031%)
Epoch: [0][100/391]	Time 0.055 (0.056)	Data 0.002 (0.008)	Loss 1.8671 (2.0559)	Prec 30.469% (21.047%)
Epoch: [0][200/391]	Time 0.051 (0.055)	Data 0.002 (0.005)	Loss 1.7032 (1.9121)	Prec 39.062% (27.247%)
Epoch: [0][300/391]	Time 0.051 (0.052)	Data 0.002 (0.004)	Loss 1.4083 (1.8174)	Prec 46.875% (31.341%)
Validation starts
Test: [0/79]	Time 0.354 (0.354)	Loss 1.5558 (1.5558)	Prec 42.188% (42.188%)
 * Prec 41.410% 
best acc: 41.410000
Epoch: [1][0/391]	Time 0.428 (0.428)	Data 0.362 (0.362)	Loss 1.4099 (1.4099)	Prec 43.750% (43.750%)
Epoch: [1][100/391]	Time 0.048 (0.052)	Data 0.002 (0.006)	Loss 1.3916 (1.4359)	Prec 50.000% (47.223%)
Epoch: [1][200/391]	Time 0.040 (0.051)	Data 0.001 (0.004)	Loss 1.4276 (1.3768)	Prec 46.094% (49.938%)
Epoch: [1][300/391]	Time 0.050 (0.049)	Data 0.002 (0.003)	Loss 1.1543 (1.3324)	Prec 60.156% (51.744%)
Validation starts
Test: [0/79]	Time 0.426 (0.426)	Loss 1.38

Epoch: [15][200/391]	Time 0.054 (0.056)	Data 0.002 (0.004)	Loss 0.5135 (0.5601)	Prec 83.594% (80.496%)
Epoch: [15][300/391]	Time 0.062 (0.053)	Data 0.002 (0.004)	Loss 0.4505 (0.5562)	Prec 82.812% (80.721%)
Validation starts
Test: [0/79]	Time 0.483 (0.483)	Loss 0.6223 (0.6223)	Prec 80.469% (80.469%)
 * Prec 78.700% 
best acc: 78.700000
Epoch: [16][0/391]	Time 0.455 (0.455)	Data 0.393 (0.393)	Loss 0.5377 (0.5377)	Prec 84.375% (84.375%)
Epoch: [16][100/391]	Time 0.039 (0.049)	Data 0.001 (0.006)	Loss 0.4802 (0.5335)	Prec 83.594% (81.706%)
Epoch: [16][200/391]	Time 0.051 (0.049)	Data 0.002 (0.004)	Loss 0.4999 (0.5435)	Prec 80.469% (81.285%)
Epoch: [16][300/391]	Time 0.051 (0.049)	Data 0.002 (0.003)	Loss 0.4867 (0.5454)	Prec 78.125% (81.206%)
Validation starts
Test: [0/79]	Time 0.281 (0.281)	Loss 0.5143 (0.5143)	Prec 82.812% (82.812%)
 * Prec 78.900% 
best acc: 78.900000
Epoch: [17][0/391]	Time 0.479 (0.479)	Data 0.390 (0.390)	Loss 0.6850 (0.6850)	Prec 77.344% (77.344%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.050 (0.047)	Data 0.001 (0.003)	Loss 0.4753 (0.4221)	Prec 82.031% (85.395%)
Validation starts
Test: [0/79]	Time 0.518 (0.518)	Loss 0.4748 (0.4748)	Prec 84.375% (84.375%)
 * Prec 81.740% 
best acc: 81.740000
Epoch: [31][0/391]	Time 0.419 (0.419)	Data 0.356 (0.356)	Loss 0.4778 (0.4778)	Prec 85.938% (85.938%)
Epoch: [31][100/391]	Time 0.051 (0.055)	Data 0.002 (0.006)	Loss 0.4700 (0.4069)	Prec 85.156% (85.999%)
Epoch: [31][200/391]	Time 0.051 (0.053)	Data 0.002 (0.004)	Loss 0.4383 (0.4068)	Prec 81.250% (85.669%)
Epoch: [31][300/391]	Time 0.066 (0.053)	Data 0.003 (0.003)	Loss 0.3968 (0.4116)	Prec 85.156% (85.501%)
Validation starts
Test: [0/79]	Time 0.364 (0.364)	Loss 0.4586 (0.4586)	Prec 85.156% (85.156%)
 * Prec 81.140% 
best acc: 81.740000
Epoch: [32][0/391]	Time 1.215 (1.215)	Data 1.167 (1.167)	Loss 0.4458 (0.4458)	Prec 85.938% (85.938%)
Epoch: [32][100/391]	Time 0.051 (0.060)	Data 0.002 (0.014)	Loss 0.4414 (0.3949)	Prec 82.812% (86.054%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.507 (0.507)	Loss 0.4075 (0.4075)	Prec 85.156% (85.156%)
 * Prec 82.230% 
best acc: 83.530000
Epoch: [46][0/391]	Time 0.462 (0.462)	Data 0.401 (0.401)	Loss 0.4497 (0.4497)	Prec 85.156% (85.156%)
Epoch: [46][100/391]	Time 0.052 (0.054)	Data 0.002 (0.006)	Loss 0.4244 (0.3693)	Prec 84.375% (86.897%)
Epoch: [46][200/391]	Time 0.051 (0.053)	Data 0.002 (0.004)	Loss 0.2887 (0.3634)	Prec 90.625% (87.177%)
Epoch: [46][300/391]	Time 0.050 (0.052)	Data 0.002 (0.004)	Loss 0.4226 (0.3679)	Prec 85.156% (87.087%)
Validation starts
Test: [0/79]	Time 0.500 (0.500)	Loss 0.3780 (0.3780)	Prec 85.156% (85.156%)
 * Prec 81.600% 
best acc: 83.530000
Epoch: [47][0/391]	Time 0.515 (0.515)	Data 0.455 (0.455)	Loss 0.2599 (0.2599)	Prec 89.062% (89.062%)
Epoch: [47][100/391]	Time 0.056 (0.057)	Data 0.002 (0.007)	Loss 0.2829 (0.3643)	Prec 92.969% (87.260%)
Epoch: [47][200/391]	Time 0.042 (0.055)	Data 0.002 (0.004)	Loss 0.4781 (0.3607)	Prec 82.812% (87.310%)
Epoch: [47][300/391]	

 * Prec 83.240% 
best acc: 84.470000
Epoch: [61][0/391]	Time 0.493 (0.493)	Data 0.438 (0.438)	Loss 0.2896 (0.2896)	Prec 90.625% (90.625%)
Epoch: [61][100/391]	Time 0.045 (0.048)	Data 0.002 (0.006)	Loss 0.3485 (0.3213)	Prec 88.281% (88.475%)
Epoch: [61][200/391]	Time 0.041 (0.045)	Data 0.001 (0.004)	Loss 0.4378 (0.3146)	Prec 87.500% (88.783%)
Epoch: [61][300/391]	Time 0.047 (0.044)	Data 0.002 (0.003)	Loss 0.4114 (0.3180)	Prec 82.031% (88.684%)
Validation starts
Test: [0/79]	Time 0.345 (0.345)	Loss 0.4112 (0.4112)	Prec 85.156% (85.156%)
 * Prec 82.720% 
best acc: 84.470000
Epoch: [62][0/391]	Time 0.516 (0.516)	Data 0.451 (0.451)	Loss 0.3013 (0.3013)	Prec 88.281% (88.281%)
Epoch: [62][100/391]	Time 0.052 (0.057)	Data 0.002 (0.006)	Loss 0.2037 (0.3014)	Prec 93.750% (89.387%)
Epoch: [62][200/391]	Time 0.060 (0.057)	Data 0.003 (0.004)	Loss 0.4131 (0.3083)	Prec 82.812% (89.121%)
Epoch: [62][300/391]	Time 0.060 (0.056)	Data 0.002 (0.003)	Loss 0.2976 (0.3151)	Prec 88.281% (88.943%)
Validation s

Epoch: [76][100/391]	Time 0.040 (0.051)	Data 0.002 (0.009)	Loss 0.3379 (0.2832)	Prec 88.281% (90.076%)
Epoch: [76][200/391]	Time 0.047 (0.048)	Data 0.002 (0.006)	Loss 0.2864 (0.2908)	Prec 91.406% (89.762%)
Epoch: [76][300/391]	Time 0.048 (0.048)	Data 0.001 (0.004)	Loss 0.3412 (0.2943)	Prec 89.062% (89.714%)
Validation starts
Test: [0/79]	Time 0.349 (0.349)	Loss 0.4253 (0.4253)	Prec 88.281% (88.281%)
 * Prec 84.250% 
best acc: 84.470000
Epoch: [77][0/391]	Time 0.538 (0.538)	Data 0.476 (0.476)	Loss 0.2067 (0.2067)	Prec 90.625% (90.625%)
Epoch: [77][100/391]	Time 0.044 (0.048)	Data 0.002 (0.007)	Loss 0.3172 (0.2900)	Prec 86.719% (90.022%)
Epoch: [77][200/391]	Time 0.043 (0.047)	Data 0.002 (0.004)	Loss 0.3211 (0.2904)	Prec 88.281% (89.914%)
Epoch: [77][300/391]	Time 0.046 (0.045)	Data 0.002 (0.004)	Loss 0.2808 (0.2958)	Prec 91.406% (89.696%)
Validation starts
Test: [0/79]	Time 0.320 (0.320)	Loss 0.3832 (0.3832)	Prec 85.938% (85.938%)
 * Prec 84.580% 
best acc: 84.580000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.050 (0.053)	Data 0.002 (0.004)	Loss 0.1434 (0.1441)	Prec 93.750% (95.149%)
Epoch: [91][300/391]	Time 0.059 (0.053)	Data 0.002 (0.003)	Loss 0.1435 (0.1424)	Prec 92.969% (95.141%)
Validation starts
Test: [0/79]	Time 0.340 (0.340)	Loss 0.2109 (0.2109)	Prec 91.406% (91.406%)
 * Prec 88.580% 
best acc: 88.630000
Epoch: [92][0/391]	Time 0.413 (0.413)	Data 0.355 (0.355)	Loss 0.2125 (0.2125)	Prec 90.625% (90.625%)
Epoch: [92][100/391]	Time 0.044 (0.048)	Data 0.002 (0.005)	Loss 0.1934 (0.1429)	Prec 92.188% (94.995%)
Epoch: [92][200/391]	Time 0.047 (0.046)	Data 0.002 (0.004)	Loss 0.0900 (0.1402)	Prec 97.656% (95.215%)
Epoch: [92][300/391]	Time 0.048 (0.045)	Data 0.002 (0.003)	Loss 0.0871 (0.1384)	Prec 98.438% (95.315%)
Validation starts
Test: [0/79]	Time 0.379 (0.379)	Loss 0.2112 (0.2112)	Prec 91.406% (91.406%)
 * Prec 88.720% 
best acc: 88.720000
Epoch: [93][0/391]	Time 0.466 (0.466)	Data 0.406 (0.406)	Loss 0.0967 (0.0967)	Prec 96.875% (96.875%)
Epoch: [93][100/391]	

In [4]:
PATH = "result/Resnet_20_quant_project/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8872/10000 (89%)



In [5]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)

7 -th layer prehooked
9 -th layer prehooked
13 -th layer prehooked
15 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
27 -th layer prehooked
34 -th layer prehooked
36 -th layer prehooked
42 -th layer prehooked
46 -th layer prehooked
48 -th layer prehooked
54 -th layer prehooked
56 -th layer prehooked
63 -th layer prehooked
65 -th layer prehooked
71 -th layer prehooked
75 -th layer prehooked
77 -th layer prehooked
83 -th layer prehooked
85 -th layer prehooked


In [ ]:
## Layer 9
## Layer 13
## save_outputs.output[1][0]
## save_outputs.output[2][0]
model.layer1[0].conv2.weight_quant.wgt_alpha

In [6]:
### Residual x0_int calculation
x0_bit = 4
w0_bit = 4
x0 = save_output.outputs[0][0]
x0_alpha = model.layer1[0].conv2.act_alpha
w0_alpha = model.layer1[0].conv2.weight_quant.wgt_alpha
x0_delta = x0_alpha/(2**x0_bit-1)
w0_delta = w0_alpha/(2**(w0_bit-1)-1)
act_quant = act_quantization(x0_bit)
x0_q = act_quant(x0,(x0_alpha*w0_alpha))
x0_int = x0_q/(x0_delta*w0_delta)

In [7]:
w_bit = 4
weight_q = model.layer1[0].conv2.weight_q
w_alpha = model.layer1[0].conv2.weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta

x_bit = 4
x = save_output.outputs[1][0]
x_alpha = model.layer1[0].conv2.act_alpha
x_delta = x_alpha/(2**x_bit-1)
act_quant = act_quantization(x_bit)
x_q = act_quant(x,x_alpha)
x_int = x_q/x_delta

conv_int = nn.Conv2d(8,8,kernel_size=3, padding=1, bias=False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
output_int = (conv_int(x_int))
psum_recovered = output_int*x_delta*w_delta + x0
relu = nn.ReLU(inplace=True)
psum_after_relu = relu(psum_recovered)
difference = (save_output.outputs[2][0] - psum_after_relu).mean()
print("The difference between psum original and psum recovered = {}".format(difference))

The difference between psum original and psum recovered = 2.9711912929997197e-07


In [8]:
act_int = x_int[0,:,:,:]  # pick only one input out of batch
# a_int.size() = [64, 32, 32]

# conv_int.weight.size() = torch.Size([64, 64, 3, 3])  <- output_ch, input_ch, ki, kj
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij

padding = 1
stride = 1
array_size = 8
x_size = x_int.size()
nig = range(x_size[2])
njg = range(x_size[3])

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

a_pad = torch.zeros((x_size[1],x_size[2]+2*padding,x_size[3]+2*padding))

a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = act_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))

ic_tile = range(int(int(act_int.size(0))/array_size))
oc_tile = range(int(int(w_int.size(0))/array_size))

a_tile = torch.zeros(len(ic_tile),array_size,len(nig)+padding*2,len(njg)+padding*2).cuda()
a_tile = torch.reshape(a_tile,(a_tile.size(0),a_tile.size(1),-1))

for ict in ic_tile:
    a_tile[ict,:,:] = a_pad[(ict*array_size):((ict+1)*array_size),:]

w_tile = torch.zeros(len(ic_tile),len(oc_tile),array_size,array_size,len(kijg)).cuda()

for oct in oc_tile:
    for ict in ic_tile:
        w_tile[ict,oct,:,:,:] = w_int[(oct*array_size):((oct+1)*array_size),(ict*array_size):((ict+1)*array_size),:]

p_nijg = range(a_pad.size(1)) ## paded activation's nij group

psum = torch.zeros(len(ic_tile),len(oc_tile),array_size,len(p_nijg),len(kijg)).cuda() 

for kij in kijg:
    for ict in ic_tile:
        for oct in oc_tile:
            for nij in p_nijg:     # time domain, sequentially given input
                m = nn.Linear(array_size, array_size, bias=False)
                m.weight = torch.nn.Parameter(w_tile[ict,oct,:,:,kij])
                psum[ict,oct,:, nij, kij] = m(a_tile[ict,:,nij]).cuda()

In [9]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32 + 2*pad = 34

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1) #34 - 2 - 1 + 1 = 32
o_nijg = range(o_ni_dim**2)    
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:  
        for ict in ic_tile:
            for oct in oc_tile:
                out[oct*array_size:(oct+1)*array_size,o_nij] = out[oct*array_size:(oct+1)*array_size,o_nij] + \
                psum[ict,oct,:, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 2nd index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

In [10]:
residual = x0_int[0,:,:,:]
residual_reshaped = torch.reshape(residual,(residual.size(0),-1))
out_after_residual = out + residual_reshaped

In [11]:
## dump all the files

## Helper functions to dump files
def dec_to_bin(arr,bit):
    bin_arr = []
    for a in arr:
        a = int(a)
        if a < 0:
            a+=bit
        b = '{0:04b}'.format(int(a))
        bin_arr.append(b)
    return bin_arr

def dec_to_bin_psum(arr,bit):
    bin_arr = []
    for a in arr:
        a = int(a)
        sign = 0
        if a < 0:
            sign = 1
            a+=bit
        b = '{0:16b}'.format(int(a))
        if sign == 1:
            b = b.replace(" ", "1")
        else:
            b = b.replace(" ","0")
        bin_arr.append(b)
    return bin_arr


def convert_to_list(arr):
    return arr.tolist()

In [12]:
## Activation dump

fp_act = open('Resnet_activation_project.txt','w')
fp_act.write("#####\n")
fp_act.write("#####\n")
fp_act.write("#####\n")
fp_act_dec = open('Resnet_activation_dec.txt','w')
for i in range(a_pad.size()[1]):
    act_line = a_pad[:,i]
    act_arr = [int(j+0.001) for j in convert_to_list(act_line)]
    for a in act_arr[::-1]:
        #print(int(a))
        fp_act_dec.write(str(int(a)))
        fp_act_dec.write(" ")
    fp_act_dec.write("\n")
    bin_act = dec_to_bin(act_arr,16)
    for b in bin_act[::-1]:
        #print(b)
        fp_act.write(b)
    fp_act.write('\n')
    

fp_act.close()
fp_act_dec.close()

In [13]:
## Weight dump
fp_wgt = open('Resnet_weight_project.txt','w')
fp_wgt.write("#####\n")
fp_wgt.write("#####\n")
fp_wgt.write("#####\n")
fp_wgt_dec = open('Resnet_weight_dec.txt','w')

for kij in range(9):
    for w in range(8):
        w_line = w_int[w,:,kij]
        w_arr = []
        for i in convert_to_list(w_line):
            if i < 0:
                w_arr.append(int(i-0.001))
            else:
                w_arr.append(int(i+0.001))
        
        for ww in w_arr[::-1]:
            #print(int(ww))
            fp_wgt_dec.write(str(int(ww)))
            fp_wgt_dec.write(" ")
        fp_wgt_dec.write("\n")
        bin_wgt = dec_to_bin(w_arr,16)
        for b in bin_wgt[::-1]:
            #print("Count = {} , b = {}".format(count,b))
            fp_wgt.write(b)
        fp_wgt.write('\n')

fp_wgt.close()
fp_wgt_dec.close()

In [14]:
## psum dump and output.txt dump
fp_psum = open('Resnet_psum_project.txt','w')
fp_psum.write("#####\n")
fp_psum.write("#####\n")
fp_psum.write("#####\n")

fp_psum_dec = open('Resnet_psum_dec.txt','w')
fp_psum_relu = open('Resnet_output_project.txt','w')
fp_psum_relu_dec = open('Resnet_output_project_dec.txt','w')
fp_psum_relu.write("#\n")
fp_psum_relu.write("#\n")
fp_psum_relu.write("#\n")
for kij in range(o_ni_dim*o_ni_dim):
    psum_line = out_after_residual[:,kij]
    psum_old = out[:,kij]
    psum_arr = []
    out_arr = []
    for i in convert_to_list(psum_line):
        if i < 0:
            #psum_arr.append(int(i-0.001))
            out_arr.append(0)
        else:
            #psum_arr.append(int(i+0.001))
            out_arr.append(int(i+0.001))
    for i in convert_to_list(psum_old):
        if i < 0:
            psum_arr.append(int(i-0.001))
        else:
            psum_arr.append(int(i+0.001))
    for p in psum_arr:
        #print(int(a))
        fp_psum_dec.write(str(int(p)))
        fp_psum_dec.write(" ")
    fp_psum_dec.write("\n")
    
    for p in out_arr:
        fp_psum_relu_dec.write(str(int(p)))
        fp_psum_relu_dec.write(" ")
    fp_psum_relu_dec.write('\n')
                               
    bin_act = dec_to_bin_psum(psum_arr,65536)
    out_bin_act = dec_to_bin_psum(out_arr,65536)
    for b in bin_act:
        #print(b)
        fp_psum.write(b)
    fp_psum.write('\n')
    
    for b in out_bin_act:
        fp_psum_relu.write(b)
    fp_psum_relu.write("\n")
fp_psum.close()
fp_psum_dec.close()
fp_psum_relu.close()
fp_psum_relu_dec.close()

In [15]:
### Residual file dump
fp_act = open('Resnet_residual_project.txt','w')
fp_act.write("#####\n")
fp_act.write("#####\n")
fp_act.write("#####\n")
fp_act_dec = open('Resnet_residual_dec.txt','w')
for i in range(o_ni_dim*o_ni_dim):
    act_line = residual_reshaped[:,i]
    act_arr = [int(j+0.001) for j in convert_to_list(act_line)]
    for a in act_arr:
        #print(int(a))
        fp_act_dec.write(str(int(a)))
        fp_act_dec.write(" ")
    fp_act_dec.write("\n")
    bin_act = dec_to_bin(act_arr,16)
    for b in bin_act:
        #print(b)
        fp_act.write(b)
    fp_act.write('\n')
    

fp_act.close()
fp_act_dec.close()

In [ ]:
## For input activations tiling. Handled in testbench. Written here just for reference logic
hardware_ni_dim = 6
hor_step = hardware_ni_dim-kernel_dim+1 ## 4
ver_step = (hardware_ni_dim-kernel_dim+1)*a_pad_ni_dim ## 136
stop_point = (a_pad_ni_dim-hardware_ni_dim)*a_pad_ni_dim+1 ## 953
group_count = 0
act_arr = []
## Vertical movement loop
for v in range(0,stop_point,ver_step):
    ## Now move horizontally
    for h in range(v,v+34,hor_step):
        if h+hardware_ni_dim>v+a_pad_ni_dim:
            break
        group = []
        group_count+=1
        for hh in range(h,h+(hardware_ni_dim)*a_pad_ni_dim,a_pad_ni_dim):
            
            for hhh in range(hardware_ni_dim):
                group.append(hh+hhh)
        act_group.append(group)       
        

In [ ]:
## For output tiling. Handled in testbench. Written here just for reference logic
our_out_hw = 4
out_ni_dim = int(math.sqrt(out.size(1)))
out_stop_point = (out_ni_dim-our_out_hw)*out_ni_dim+1
out_ver_step = our_out_hw*out_ni_dim
group_count = 0
out_group = []
for v in range(0,out_stop_point,out_ver_step):
    ## Now move horizontally
    for h in range(v,v+32,our_out_hw):
        group = []
        group_count+=1
        for hh in range(h,h+our_out_hw*out_ni_dim,out_ni_dim):
            for hhh in range(our_out_hw):
                group.append(hh+hhh)
        out_group.append(group)
